In [1]:
from gibbsml.ellingham.ellingham_backend import Ellingham

USER_API_KEY = ''  # Type your own Materials Project API key. See https://materialsproject.org/open 

## 1. Model set up.

#### The Elligham python class contains a previously trained model for predicting the free-energy for the formation of metal oxides (binary and ternary compounds). 

#### The user needs to introduce the chemical formula of the compound for the predictions that is interested in. For instance, in the following example we would like to predict the free-energy for the formation of LiAlO2 from their elements (Li and Al).

In [32]:
compound = 'LiAlO2'  # Name of the compound to get the predictions.
ellingham = Ellingham(USER_API_KEY=USER_API_KEY,
                      id_mo=compound)

Getting information for LiAlO2
Fingerprint for LiAlO2_mp-3427 completed.


#### The user can alternatively specify the Materials Project ID (mp-id) of the metals participating in the reaction along with the metal oxide mp-id:

In [33]:
id_mo = 'mp-3427'  # LiAlO2 mp-id
id_m1 = 'mp-135'  # Li mp-id
id_m2 = 'mp-134'    # Al mp-id.
ellingham = Ellingham(USER_API_KEY=USER_API_KEY,
                      id_mo=id_mo, id_m1=id_m1, id_m2=id_m2)

Getting information for mp-3427
Fingerprint for LiAlO2_mp-3427 completed.


## 2. Get predictions.

#### We can ask for the predicted formation energy and entropy:

In [27]:
pred_dH0 = ellingham.get_dH0()
pred_dS0 = ellingham.get_dS0()
print('Predicted formation energy ( 0 K ):', pred_dH0, 'kJ/mol')
print('Predicted entropy:', pred_dS0, 'kJ/mol')

Predicted formation energy ( 0 K ): -1196.4137925572495 kJ/mol
Predicted entropy: 0.22641729206407463 kJ/mol


#### We can also ask for the predicted free-energy at a given temperature (T):

In [28]:
T = 298.15  # temperature in K.
pred_dG0 = ellingham.get_dG0(T=T)
print('Predicted free-energy (', T, 'K):', pred_dG0, 'kJ/mol')

Predicted free-energy ( 298.15 K): -1128.9074769283457 kJ/mol


#### The balanced reaction equation can be also accessed using:

In [29]:
print('Balanced reaction:')
print(ellingham.get_balanced_reaction())

Balanced reaction:
1.0 Li + 1.0 Al + O2 --> 1.0 LiAlO2


## 3. Plot Ellingham diagram.

In [65]:
import plotly.graph_objects as go
import numpy as np

def get_dG0_CO_expt(T):
    dG0_CO_expt = -2.23623129e+02 -1.75809004e-01 * T  # Experimental free-energy of CO from the NIST-JANAF database. Malcolm W. Chase, Jr. NIST-JANAF Thermochemical Tables. Washington, DC : New York :American Chemical Society ; American Institute of Physics for the National Institute of Standards and Technology, 1998.
    return dG0_CO_expt

def get_dG0_LiAlO2_expt(T):
    dG0_LiAlO2_expt = -1.19217958e+03 + 2.28562409e-01 * T  # Experimental free-energy for LiAlO2.
    return dG0_LiAlO2_expt

T_range = np.arange(0, 3001, 1)  # Temperature range, from 0K to 3000K.

dG0_MO = []
dG0_CO = []
dG0_MO_expt = []

for i in T_range:
    dG0_CO.append(get_dG0_CO_expt(T=i))
    dG0_MO.append(ellingham.get_dG0(T=i))
    dG0_MO_expt.append(get_dG0_LiAlO2_expt(T=i))


# Plots.
fig = go.Figure()
fig.add_trace(go.Scatter(x=T_range, y=dG0_CO, name="C(s) + O2 --> CO", line=dict(color='black', width=2)))  # CO.
fig.add_trace(go.Scatter(x=T_range, y=dG0_MO_expt, name="Li + Al + O2 --> LiAlO2 (expt)", line=dict(color='blue', width=2)))  # MO.
fig.add_trace(go.Scatter(x=T_range, y=dG0_MO, name="Li + Al + O2 --> LiAlO2 (pred)", line=dict(color='red', width=2)))  # MO.


fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))  # Legend.

fig.update_layout(
    xaxis_title="Temperature (K)",
    yaxis_title="Free-energy (kJ/mol)",
    font=dict(
        size=18,
    )
)

fig.show()
